# [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation)  
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [ ]:
## 1. 모듈 import
from dotenv import load_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import trim_messages
from langchain_core.chat_history import (
    BaseChatMessageHistory, 
    InMemoryChatMessageHistory
)


## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
## type: dict 
store = {}

## 4. 함수 정의: 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 6. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


## 7. trim 설정
trimmer = trim_messages(
    max_tokens = 60,
    strategy = 'last',
    include_system = True,
    token_counter = llm,
    allow_partial = False,
    start_on = 'human',
)

## 8. chain 구성
chain = ( RunnablePassthrough.assign(chat_history = itemgetter('chat_history') | trimmer)
    | prompt 
    | llm )

## 9. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

## 10. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '1234'}}      
    )

이력서 작성 시, 먼저 자기소개부터 시작하는 것이 좋습니다. 이름, 연락처, 학력, 경력 등을 포함한 기본 정보를 명확하게 작성해야 해요. 이후에는 자신의 강점과 경험을 부각할 수 있는 섹션을 넣어야 해요. 경력이 있다면 해당 경험을 상세히 기술하고, 신입자라면 학창 시절의 프로젝트나 아르바이트 경일반적으로 이력서는 이메일을 통해 제출하거나 회사의 채용 포털에 온라인으로 업로드합니다. 제출 형식은 회사나 채용 정보에 따라 다르므로 필요 시 지원 공고를 참고하시기 바랍니다. 이메일로 제출할 경우는 제목에 '지원자 성함 - 지원 직무'와 같이 표기하고 본문에 간단한 자기소개와 이력서 파일을 첨부합니다. 온라인

In [ ]:
for info in get_history('1234').messages:
    print(f'{[info.type.upper()]}: {info.content}\n'),

['HUMAN']: 이력서 작성은 어떻게 해?

['AI']: 이력서 작성 시, 먼저 자기소개부터 시작하는 것이 좋습니다. 이름, 연락처, 학력, 경력 등을 포함한 기본 정보를 명확하게 작성해야 해요. 이후에는 자신의 강점과 경험을 부각할 수 있는 섹션을 넣어야 해요. 경력이 있다면 해당 경험을 상세히 기술하고, 신입자라면 학창 시절의 프로젝트나 아르바이트 경

['HUMAN']: 이력서는 어떻게 제출해?

['AI']: 일반적으로 이력서는 이메일을 통해 제출하거나 회사의 채용 포털에 온라인으로 업로드합니다. 제출 형식은 회사나 채용 정보에 따라 다르므로 필요 시 지원 공고를 참고하시기 바랍니다. 이메일로 제출할 경우는 제목에 '지원자 성함 - 지원 직무'와 같이 표기하고 본문에 간단한 자기소개와 이력서 파일을 첨부합니다. 온라인

